In [ ]:
# import libraries
# idk if i actually need all these libraries
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
from skimage import exposure
from skimage.util import random_noise
from skimage import transform
import cv2
from cv2 import resize
from google.colab import drive, auth
import gspread
from google.auth import default
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import tensorflow as tf
# might have to import pandas also

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/cropped_images.zip -d /content/image_folder

Archive:  /content/drive/MyDrive/cropped_images.zip
  inflating: /content/image_folder/cropped_images/IMG0001.png  
  inflating: /content/image_folder/cropped_images/IMG0002.png  
  inflating: /content/image_folder/cropped_images/IMG0003.png  
  inflating: /content/image_folder/cropped_images/IMG0004.png  
  inflating: /content/image_folder/cropped_images/IMG0005.png  
  inflating: /content/image_folder/cropped_images/IMG0006.png  
  inflating: /content/image_folder/cropped_images/IMG0007.png  
  inflating: /content/image_folder/cropped_images/IMG0008.png  
  inflating: /content/image_folder/cropped_images/IMG0009.png  
  inflating: /content/image_folder/cropped_images/IMG0010.png  
  inflating: /content/image_folder/cropped_images/IMG0011.png  
  inflating: /content/image_folder/cropped_images/IMG0012.png  
  inflating: /content/image_folder/cropped_images/IMG0013.png  
  inflating: /content/image_folder/cropped_images/IMG0014.png  
  inflating: /content/image_folder/cropped_images/IM

MY LOGIC: have a bunch of data augmentation functions i can all, each time i import in an image i collect its associated labels (x-coor, y-coor, theta), then i augment the image like 10 different ways all with the same labels, i then append these images and their labels into the training dataset

SPLITTING DATA: I will take like one out of every 8 or 9 photos for the testing dataset. So I can modulo while reading through the csv file of all the images paths and labels to skip every 8th image when making the training dataset, and only including every 8th image when making the testing dataset

NOTES: I'm using a lot of code from https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178, so it needs to be adapted to make sense to our situation.

In [ ]:
# function add random noise to an image
def add_noise(img):
    img_noise = random_noise(img, mode = 's&p', clip=True)
    return img
    '''
    VARIABILITY = 50
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img
    '''

In [ ]:
# data augmentation function
# might have to separate out these functions seperately
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    # rotate slightly
    rotation_range=20,
    width_shift_range=0.0,
    height_shift_range=0.0,
    # deviate brightness with 0.5 from original
    brightness_range=[0.4,1.5],
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    # add function for introducing random noise
    preprocessing_function=add_noise,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1,
    dtype=None
)

In [ ]:
IM_WIDTH = 2819
IM_HEIGHT = 2085
train_generator = datagen.flow_from_directory(
    # what is the path to these files?
    directory='/content/image_folder',
    # set constants for what the width and height of all images will be
    target_size=(IM_WIDTH, IM_HEIGHT), #Image dimensions: 2819 x 2085
    color_mode="rgb", # for coloured images
    batch_size=1, # number of images to extract from folder for every batch
    # what is the class_mode?
    class_mode="binary", # classes to predict
    seed=2020 # to make the result reproducible
    )

Found 400 images belonging to 1 classes.


In [ ]:
# pre-process and augment image before inputting into the network

def preprocess_image(self, img_path, target_size=(224, 224)):
        im = Image.open('image_folder/cropped_images/'+img_path+'.png')
        im_arr = img_to_array(im)
        # do i need to expand dimensions of the image
        im = im.resize(target_size)
        # augment data
        iterator = datagen.flow(im_arr, batch_size=1)
        for i in range(9):
	      # generating images of each batch
          batch = iterator.next()
        im = np.array(im) / 255.0

        return im

In [ ]:
# an idea I found on https://stackoverflow.com/questions/69159550/keras-is-there-a-way-to-manage-the-filenames-generated-by-the-flow-from-direct
# this could help us expand our dataset first before running the model
# additionally, it would make the filenames somewhat intuitive and then i could maybe go in and edit the csv with the filenames and labels to include the filenames of the augmented images

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress Tensorflow messages
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np


image_folder = 'img_folder' # your image folder
print(f'my images: {os.listdir(image_folder)}')


aug_image_folder = 'aug_img_folder' # aug images will be saved here
os.makedirs(aug_image_folder,exist_ok=True)


# define your augmentations
aug = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    # rotate slightly
    rotation_range=20,
    width_shift_range=0.0,
    height_shift_range=0.0,
    # deviate brightness with 0.5 from original
    brightness_range=[0.4,1.5],
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    # add function for introducing random noise
    preprocessing_function=add_noise,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1,
    dtype=None)


# iterate over the image folder
image_names = os.listdir(image_folder)
for image_name in image_names:
    img = cv2.imread(f'{image_folder}/{image_name}',1)

    # get the augmented image
    aug_img_iterator = aug.flow(x=np.expand_dims(img,0),batch_size=1)
    aug_img=next(aug_img_iterator)

    # should i repeat this multiple times to get multiple augmented images
    # an example of for loop: https://medium.com/featurepreneur/image-data-augmentation-using-keras-imagedatagenerator-1cee60255ea8

    # save the augmented image
    cv2.imwrite(f'{aug_image_folder}/{image_name}',aug_img[0,:,:,:])

print(f'aug images: {os.listdir(aug_image_folder)}')

FileNotFoundError: [Errno 2] No such file or directory: 'img_folder'

In [ ]:
from keras.utils import to_categorical
from PIL import Image

In [ ]:
class UtkFaceDataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """

    def __init__(self, df):
        self.df = df


    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        TRAIN_TEST_SPLIT = 0.8
        # determine constants for splitting training and testing datasets
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]

        # converts alias to id
        # this should be x, y, and theta
        # converted all to "max" assingment on account of continuous numerical parameters
        self.max_x = self.df['x_coor'].max()
        self.max_y = self.df['y_coor'].max()
        self.max_theta = self.df['theta'].max()

        return train_idx, valid_idx, test_idx

    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open('image_folder/cropped_images/'+img_path+'.png')
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im_arr = img_to_array(im)
        # do i need to expand dimensions of the image
        im_arr = expand_dims(im_arr, 0)
        # add data augmentation to image preprocessing
        iterator = datagen.flow(im_arr, batch_size=1)
        for i in range(9):
	        # generating images of each batch
          batch = iterator.next()
        im = np.array(im) / 255.0

        # return im1, im2, im3, etc.
        return im

    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """

        # arrays to store our batched data
        images, xs, ys, thetas = [], [], [], []
        while True:
            for idx in image_idx:
                image = self.df.iloc[idx]

                # how do i parse the data with labels?
                x = image['x_coor']
                y = image['y_coor']
                theta = image['theta']
                file = image['img_path']

                # augment to a list of images, and unfold the images
                # im1, im2, im3, etc. = self.preprocess_image(file)
                im = self.preprocess_image(file)

                # ASK: what is this?
                # ANSWER: This is the update to the dictionary wih scaled ages, and it also selects which race and gender category a face should fall into.
                xs.append(x)
                ys.append(y)
                thetas.append(theta)
                images.append(im)

                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(xs), np.array(ys), np.array(thetas)]
                    images, xs, ys, thetas = [], [], [], []

            if not is_training:
                break

In [ ]:
# loading data
# how can i integrate this with above function?

import pandas as pd

# my training data
train_data_df = pd.read_csv('/content/CSE571DataCollection.csv')
train_data_df = train_data_df[['img_path','x_coor', 'y_coor', 'theta']]
#shuffle dataset
train_data_df = train_data_df.sample(frac=1)

# my testing data
# test_data_df = pd.read_csv('test_set.csv')
# test_data_df = test_data_df[['img_path','x_coor', 'y_coor', 'theta']]

In [ ]:
# rename this to make more sense
data_generator = UtkFaceDataGenerator(train_data_df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

In [ ]:
# some of these libraries may be repeated
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input

In [ ]:
class LocalizationMultiOutputModel():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:

        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        return x

    def build_x_coor_branch(self, inputs):
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        # alter the architecture of neural network as i see fit
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="x_coor_output")(x)
        return x

    def build_y_coor_branch(self, inputs):
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="y_coor_output")(x)
        return x

    def build_theta_branch(self, inputs):
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="theta_output")(x)
        return x

    def assemble_full_model(self, width, height):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 3)
        # what is my input_shape?
        inputs = tf.keras.layers.Input(shape=input_shape)
        x_coor_branch = self.build_x_coor_branch(inputs)
        y_coor_branch = self.build_y_coor_branch(inputs)
        theta_branch = self.build_theta_branch(inputs)
        model = Model(inputs=inputs,
                     outputs = [x_coor_branch, y_coor_branch, theta_branch],
                     name="localization_net")
        return model

In [ ]:
IM_WIDTH = 224
IM_HEIGHT = 224
model = LocalizationMultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT)

In [ ]:
# training the model

from keras.optimizers import Adam
init_lr = 1e-4
epochs = 20
opt = Adam(learning_rate=init_lr, weight_decay=init_lr / epochs)
model.compile(optimizer=opt,
              loss={
                  'x_coor_output': 'mse',
                  'y_coor_output': 'mse',
                  'theta_output': 'mse'},
              loss_weights={
                  # should i weight the outputs differently?
                  'x_coor_output': 4.,
                  'y_coor_output': 4.,
                  'theta_output': 4.},
              metrics={
                  'x_coor_output': 'mae',
                  'y_coor_output': 'mae',
                  'theta_output': 'mae'})

In [ ]:
# using a ModelCheckpoint callback in order to save the model on disk at the end of each epoch.

from keras.callbacks import ModelCheckpoint
batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]
history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

<ipython-input-111-3cc7e7b02e26>:11: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/20
8/8 [==============================] - 114s 15s/step - loss: 48.5369 - x_coor_output_loss: 4.5349 - y_coor_output_loss: 3.6435 - theta_output_loss: 3.9558 - x_coor_output_mae: 1.6554 - y_coor_output_mae: 1.5136 - theta_output_mae: 1.5799 - val_loss: 4.4449 - val_x_coor_output_loss: 0.3880 - val_y_coor_output_loss: 0.4496 - val_theta_output_loss: 0.2736 - val_x_coor_output_mae: 0.5521 - val_y_coor_output_mae: 0.5616 - val_theta_output_mae: 0.4262
Epoch 2/20
8/8 [==============================] - 88s 13s/step - loss: 39.9843 - x_coor_output_loss: 3.7817 - y_coor_output_loss: 2.9423 - theta_output_loss: 3.2721 - x_coor_output_mae: 1.5847 - y_coor_output_mae: 1.3033 - theta_output_mae: 1.4456 - val_loss: 3.9625 - val_x_coor_output_loss: 0.3978 - val_y_coor_output_loss: 0.3556 - val_theta_output_loss: 0.2372 - val_x_coor_output_mae: 0.5593 - val_y_coor_output_mae: 0.4779 - val_theta_output_mae: 0.3874
Epoch 3/20
8/8 [==============================] - 87s 12s/step - loss: 44.8267 

KeyboardInterrupt: 

In [ ]:
# calculate mean absolute error for age

import plotly.graph_objects as go

print(history.history)

plt.clf()
fig = go.Figure()
fig.add_trace(go.Scattergl(
                    y=history.history['x_coor_output_mae'],
                    name='X Coordinate Train'))
fig.add_trace(go.Scattergl(
                    y=history.history['val_x_coor_output_mae'],
                    name='X Coordinate Valid'))
fig.add_trace(go.Scattergl(
                    y=history.history['y_coor_output_mae'],
                    name='Y Coordinate Train'))
fig.add_trace(go.Scattergl(
                    y=history.history['val_y_coor_output_mae'],
                    name='Y Coordinate Valid'))
fig.add_trace(go.Scattergl(
                    y=history.history['theta_output_mae'],
                    name='Theta Train'))
fig.add_trace(go.Scattergl(
                    y=history.history['val_theta_output_mae'],
                    name='Theta Valid'))
fig.update_layout(height=500,
                  width=700,
                  title='Mean Absolute Error',
                  xaxis_title='Epoch',
                  yaxis_title='Mean Absolute Error')
fig.show()

{'loss': [69.18416595458984, 57.70153045654297, 41.60118103027344, 33.608612060546875, 33.70928192138672, 34.30934524536133, 30.098119735717773, 29.728015899658203, 31.015766143798828, 32.66541290283203], 'x_coor_output_loss': [5.525417804718018, 4.723474979400635, 3.170382022857666, 3.1152875423431396, 2.6688454151153564, 2.644390344619751, 2.656175136566162, 2.212505578994751, 2.243044376373291, 2.6792783737182617], 'y_coor_output_loss': [6.077456951141357, 4.221012592315674, 3.0802063941955566, 2.6799557209014893, 2.386545419692993, 2.8164494037628174, 2.5447840690612793, 2.8762125968933105, 2.614772319793701, 2.4456374645233154], 'theta_output_loss': [5.693166255950928, 5.480895042419434, 4.1497063636779785, 2.606909990310669, 3.3719308376312256, 3.1164968013763428, 2.323570489883423, 2.343285322189331, 2.896124839782715, 3.041438579559326], 'x_coor_output_mae': [1.8481380939483643, 1.6870081424713135, 1.3631099462509155, 1.3446170091629028, 1.2508454322814941, 1.2544960975646973, 

<Figure size 640x480 with 0 Axes>

In [ ]:
# calculate overall loss
# ASK: how is this evaluated?

fig = go.Figure()
fig.add_trace(go.Scattergl(
                    y=history.history['loss'],
                    name='Train'))
fig.add_trace(go.Scattergl(
                    y=history.history['val_loss'],
                    name='Valid'))
fig.update_layout(height=500,
                  width=700,
                  title='Overall Loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()